In [1]:
pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 4.6 MB/s 
     |████████████████████████████████| 1.0 MB 36.2 MB/s 


In [7]:
# Dataset analysis
import pandas as pd 
from pyvi.ViTokenizer import ViTokenizer

DATA = 'drive/My Drive/CODE/ViSocial/dataset/hsd/aug/train_augmented_dataset_2.csv'
# DATA = 'drive/My Drive/CODE/ViSocial/dataset/hsd/train.csv'

data = pd.read_csv(DATA, index_col=False)

label0 = data.loc[data['label_id']==0]
label1 = data.loc[data['label_id']==1]
label2 = data.loc[data['label_id']==2]

print("Total data:", len(data))
print("Total data in label 0: ", len(label0))
print("Total data in label 1: ", len(label1))
print("Total data in label 2: ", len(label2))

def get_total_words(dt):
    texts = dt['free_text']
    count = 0
    for t in texts:
        tokenized_t = ViTokenizer.tokenize(str(t))
        list_words = tokenized_t.split()
        count = count + len(list_words)

    return count


print("Vocabulary size in labels 0:", get_total_words(label0))
print("Vocabulary size in labels 1:", get_total_words(label1))
print("Vocabulary size in labels 2:", get_total_words(label2))
print("Total Vocabulary size:", get_total_words(data))

print("Average words in labels 0:", get_total_words(label0)/len(label0))
print("Average words in labels 1:", get_total_words(label1)/len(label1))
print("Average words in labels 2:", get_total_words(label2)/len(label2))
print("Average words:", get_total_words(data)/len(data))

Total data: 47025
Total data in label 0:  19886
Total data in label 1:  11287
Total data in label 2:  15852
Vocabulary size in labels 0: 208398
Vocabulary size in labels 1: 99918
Vocabulary size in labels 2: 217880
Total Vocabulary size: 526196
Average words in labels 0: 10.479633913305843
Average words in labels 1: 8.85248515991849
Average words in labels 2: 13.744637900580368
Average words: 11.189707602339181


In [ ]:
# Filter label 1,2 - train

# Dataset analysis
import pandas as pd 

DATA = 'drive/My Drive/CODE/ViSocial/dataset/hsd/train.csv'
DATA_HATE = 'drive/My Drive/CODE/ViSocial/dataset/hsd/aug/data_label_1_2.csv'

data = pd.read_csv(DATA, index_col=False)

label1 = data.loc[data['label_id']==1]
label2 = data.loc[data['label_id']==2]

data_new = pd.concat([label1, label2])

print(data_new)
data_new.to_csv(DATA_HATE, header=False, index=False, sep="|")

                                               free_text  label_id
7                                      Lúp lúp như chó .         1
18     Dạy bơi cho cá. Bơi thì đương nhiên nó bơi đượ...         1
38              Ý thức còn ít hơn cả số tiền trong túi t         1
47                                           xxx video 🔞         1
58                                 Đấu khẩu - Chim lợn 👍         1
...                                                  ...       ...
23950                                 Dũng thằng xàm lol         2
23961                            Bổ chết moẹ abe bây giờ         2
24013  Công nghệ của cs đã đạt mức thượng thừa rồi, c...         2
24016  Corona ăn mất cái ý thức của chị rồi hả chị ơi :)         2
24030  Nói tóm lại là thời gian này A chung đang bị n...         2

[4162 rows x 2 columns]


In [3]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/CODE/ViSocial/cc.vi.300.vec',binary=False)

In [ ]:
# model.most_similar("lũ")

In [ ]:
# model.most_similar("lũ")

In [ ]:
# random.randint(3, 9)

In [5]:
# TRAIN 
# CODE augmetation: https://github.com/jasonwei20/eda_nlp

import random
from random import shuffle

random.seed(1)
import json

# import nltk

# nltk.download('wordnet')

class Argument:
    input = "drive/My Drive/CODE/ViSocial/dataset/hsd/aug/data_label_1_2.csv"
    output = "drive/My Drive/CODE/ViSocial/dataset/hsd/aug/augmented_dataset_2.txt"
    num_aug = 10
    alpha = 0.1


args = Argument()

# stop words list
stop_words = []
# with open("drive/My Drive/CODE/HSD/vietnamese-stopwords-dash.txt", "r") as f:
#     stop_words = []
#     for line in f:
#         dd = line.strip('\n')
#         stop_words.append(dd)

# cleaning up text
import re


def get_only_chars(line):
    # clean_line = ""

    # line = line.replace("’", "")
    # line = line.replace("'", "")
    # line = line.replace("-", " ") #replace hyphens with spaces
    # line = line.replace("\t", " ")
    # line = line.replace("\n", " ")
    # line = line.lower()

    # for char in line:
    #     if char in 'qwertyuiopasdfghjklzxcvbnm ':
    #         clean_line += char
    #     else:
    #         clean_line += ' '

    # clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    # if clean_line[0] == ' ':
    #     clean_line = clean_line[1:]
    # return clean_line
    return line


########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################

# for the first time you use wordnet
# import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet


def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            # print("replaced", random_word, "with", synonym)
            num_replaced += 1
        if num_replaced >= n:  # only replace up to n words
            break

    # this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words


# def get_synonyms(word):
# 	synonyms = set()
# 	for syn in wordnet.synsets(word):
# 		for l in syn.lemmas():
# 			synonym = l.name().replace("_", " ").replace("-", " ").lower()
# 			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
# 			synonyms.add(synonym)
# 	if word in synonyms:
# 		synonyms.remove(word)
# 	return list(synonyms)

# def get_synonyms(word):
#     synonyms = set()
#     with open("drive/My Drive/CODE/HSD/word_net_vi.json", "r") as f:
#         wordnet = json.load(f)

#     for key, value in wordnet.items():
#         if key.strip() == word:
#             for v in value:
#                 synonyms.add(v.strip())

#         if word in synonyms:
#             synonyms.remove(word)
#     return list(synonyms)

def get_synonyms(word):
    synonyms = []
    sn = model.most_similar(word)

    for k in sn:
        synonyms.append(k[0])
    return synonyms


########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

def random_deletion(words, p):
    # obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    # randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    # if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words) - 1)
        return [words[rand_int]]

    return new_words


########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        if len(new_words) > 0:
            new_words = swap_word(new_words)
    return new_words


def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words) - 1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words) - 1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words


########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words


def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1 and len(new_words) > 0:
    # while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words) - 1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    
    if len(new_words) > 0:
        random_synonym = synonyms[0]
        random_idx = random.randint(0, len(new_words) - 1)
        new_words.insert(random_idx, random_synonym)


########################################################################
# main data augmentation function
########################################################################

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not '']
    num_words = len(words)

    augmented_sentences = []

    if len(words) <= 0:
        return augmented_sentences
    num_new_per_technique = int(num_aug / 4) + 1
    n_sr = max(1, int(alpha_sr * num_words))
    n_ri = max(1, int(alpha_ri * num_words))
    n_rs = max(1, int(alpha_rs * num_words))

    # sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    # ri
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = list(set(augmented_sentences))
    augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    shuffle(augmented_sentences)

    # trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    # append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences


# the output file
output = None
if args.output:
    output = args.output
else:
    from os.path import dirname, basename, join

    output = join(dirname(args.input), 'eda_' + basename(args.input))

# number of augmented sentences to generate per original sentence
num_aug = 9  # default
if args.num_aug:
    num_aug = args.num_aug

# how much to change each sentence
alpha = 0.1  # default
if args.alpha:
    alpha = args.alpha


# generate more data with standard augmentation
def gen_eda(train_orig, output_file, alpha, num_aug=9):
    try:
        writer = open(output_file, 'w')
        lines = open(train_orig, 'r').readlines()

        writer.write("free_text" + "," + "label_id" + '\n')
        augm = ""
        for i, line in enumerate(lines):
            try:
                parts = line[:-1].split('|')
                # print(parts)
                # sen_id = parts[0]
                label = parts[1]
                sentence = parts[0]
                aug_sentences = eda(sentence, alpha_sr=alpha, alpha_ri=alpha, alpha_rs=alpha, p_rd=alpha, num_aug=num_aug)
                for aug_sentence in aug_sentences:
                    # writer.write(label + "\t" + aug_sentence + '\n')
                    # writer.write(sen_id + "," +aug_sentence + "," + label + '\n')
                    augm = augm + aug_sentence + "," + label + '\n'
            except Exception as e:
                print(e)
                print(parts)
                pass

        writer.write(augm)
        writer.close()
        print(
            "generated augmented sentences with eda for " + train_orig + " to " + output_file + " with num_aug=" + str(
                num_aug))
    except Exception as e:
        raise e
        pass


# main function
if __name__ == "__main__":
    # generate augmented sentences and output into a new file
    gen_eda(args.input, args.output, alpha=alpha, num_aug=num_aug)


"word 'chó,' not in vocabulary"
['Sống như chó, nhà nhưng thích tuyên truyền.', '2']
"word 'Dcmm' not in vocabulary"
['Dcmm cmt bài viết của bố ăn lozz ak cut', '2']
"word '🙂' not in vocabulary"
['Hằng Thanh vl đây là cl 🙂 hoa đ gì mà hoa 🙂', '2']
"word ':)))' not in vocabulary"
['Long Nguyen :))) câm', '2']
"word 'j.ng' not in vocabulary"
['Vâng.thằng cc nào cũng ngồi ngoài phán như thánh chờ xong rồi ms vào.chả biết cái đéo j.ng ta làm gì chúng mày biết chắc.biết bao nhiêu vụ ngăn chặn được không đưa ra ng ta báo cáo cho chúng mày à', '2']
"word 'lo.' not in vocabulary"
['kệ cha chúng mày mọi việc đã có ĐẺNG và đám BÒ ĐỎ ÓC BÒ nó lo.', '2']
"word 'bảo:' not in vocabulary"
['Và thằng cha bảo: Mày xuống xe ngay. Mất dạy', '2']
list index out of range
['"@Dad ‘S - ""Mỹ ác hơn tàu chứ bạn ơi""']
list index out of range
['Tàu ác hơn Mỹ trăm lần chứ bạn ơi']
'word \'""nó\' not in vocabulary'
['- ""có lẽ bạn kg biết lịch sử chính phủ Mỹ ...."" Có lẽ bạn không biết lịch sử tàu luôn luôn muốn

In [6]:
# TRAIN
# concat train original with original

import pandas as pd

DATA = 'drive/My Drive/CODE/ViSocial/dataset/hsd/train.csv'
# DATA = 'drive/My Drive/CODE/Hate speech detection/data/vlsp_train.csv'
DATA_AUG = 'drive/My Drive/CODE/ViSocial/dataset/hsd/aug/augmented_dataset_2.txt'

# DATA_AUG_FINAL = 'drive/My Drive/CODE/Hate speech detection/data_aug/original_augmented_dataset.csv'
DATA_AUG_FINAL = 'drive/My Drive/CODE/ViSocial/dataset/hsd/aug/train_augmented_dataset_2.csv'

data = pd.read_csv(DATA, index_col=False)
data_hate = pd.read_csv(DATA_AUG, index_col=False, error_bad_lines=False)

# data_hate = data_hate.iloc[: , 1:]

data_aug = pd.concat([data, data_hate])
data_aug.to_csv(DATA_AUG_FINAL, index=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 150: expected 2 fields, saw 3\nSkipping line 151: expected 2 fields, saw 3\nSkipping line 152: expected 2 fields, saw 3\nSkipping line 153: expected 2 fields, saw 3\nSkipping line 154: expected 2 fields, saw 3\nSkipping line 155: expected 2 fields, saw 3\nSkipping line 156: expected 2 fields, saw 3\nSkipping line 157: expected 2 fields, saw 3\nSkipping line 158: expected 2 fields, saw 3\nSkipping line 159: expected 2 fields, saw 3\nSkipping line 160: expected 2 fields, saw 3\nSkipping line 804: expected 2 fields, saw 3\nSkipping line 805: expected 2 fields, saw 3\nSkipping line 806: expected 2 fields, saw 3\nSkipping line 807: expected 2 fields, saw 3\nSkipping line 808: expected 2 fields, saw 3\nSkipping line 809: expected 2 fields, saw

In [ ]:
data_hate

,free_text,label_id
0,như là Lúp lúp như chó .,1
1,Lúp lúp như chó .,1
2,Lúp như là lúp như chó .,1
3,chó lúp như Lúp .,1
4,Lúp lúp như sau chó .,1
...,...,...
25175,Corona mất ý thức của chị hả chị ơi :),2
25176,Corona ăn choán cái ý không ngủ của chị rồi hả...,2
25177,ăn cái mất Corona ý thức của chị rồi hả chị ơi :),2
25178,Corona ăn mất cái ý thức của chị rồi hả chị ơi :),2


In [ ]:
pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.5 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3131550 sha256=33a6c23a22d3ff921ee08273b712d93554ce90260f4bfe904ac899b1bfc4bbba
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [ ]:
pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/CODE/ViSocial/cc.vi.300.vec')

In [ ]:
model.most_similar("lũ")

[('Lũ', 0.7945042848587036),
 ('lũSau', 0.6111865043640137),
 ('lủ', 0.6068373918533325),
 ('lũ', 0.6020839214324951),
 ('Rặt', 0.5421708226203918),
 ('lũMưa', 0.5227053761482239),
 ('9N2', 0.5213004350662231),
 ('lụt', 0.5200181007385254),
 ('ct12a', 0.5179020166397095),
 ('ct12', 0.5175750255584717)]

In [ ]:
import fasttext
model = fasttext.load_model('drive/MyDrive/ViSocial/cc.vi.300.vec')
model.get_nearest_neighbors('dog', k=100)


ValueError: ignored